# Words Apart
## Levenshtein Distance

## How far do you say?
### Measuring the gaps between words

Bioinformatics studies the building blocks of life. To examine a few more Python tools, we will look at the most basic building block of bioinformatics - [edit distance](https://en.wikipedia.org/wiki/Edit_distance).

This is how you can measure the separation between two strings of letters. If you remember the sci-fi film Gattica (about a world gone genetic-testing crazy), or even school biology, you may be aware that our genes are made up of long DNA sequences of nucleic acids, called _bases_ . There are only four bases: `G`, `A`, `T` and `C`. Guess where the film name comes from...

This means you can understand a lot about genetic similarity by comparing long sequences and working our how far apart they are - perhaps what genetic mutations would get from one version to another, to understand what animals, plants or bacteria are related and why they differ.

GGACTATCTACTACCATACGGACTATCTACTACCATACGGACTATCTACTACCATACGGACTATCTACTACCATACGGACTATCTACTACCATAC...
GAGCTATCTACCTAGCATTCGACTAACTACTACCATTCGGACTATCTACTACCATACGGACTATCTACTACCATACGGACTATCTACTACCATAC...

You can see these are similar, but not quite the same, but more more alike than either are to:
GGGGGGGGGGGGGGGGGGGAAAAAAAAAAAAAAAAAAAAAAAAAATTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTCCCCCCCCCCCCC...

How do we quantify that? One way:

**Levenshtein Distance**: (roughly) the minimum number of single-character edits to get from the first string to the second

In this exercise, we will touch on some of the tools that will become more foundational in the IDE setting, with VSCode.

## pip

`pip` is one of a wide range of tools constituting the Python packaging ecosystem. It is hugely fragmented compared to most languages, but `pip` is a relatively simple and standard way of installing tools. You may also come across the Anaconda distribution, and its tool `conda`, which works very similarly. We need additional libraries for this exercise - you can install them as follows:

In [ ]:
!pip install python-Levenshtein pytest-benchmark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.1 MB/s eta 0:00:00


Good practice is to have a requirements file, where version limits can be set for each package - to avoid accidental breaking upgrades (a common standard is to pin the major version number, as under semantic versioning practice, minor versions should not introduce major breaking changes). Then, when you clone down the repository, one way of installing dependencies is:

In [ ]:
!pip install -r requirements.txt

In [ ]:
!cat requirements.txt

numpy
pytest<8
pytest-benchmark==3.2.3
python-levenshtein >=0.10, <0.13


You can see here that `pytest-benchmark` and `python-levenshtein` are pinned - ideally being generous down the way, but not giving _too_ much flexibility up the way (you don't know if a breaking change will come in a dependency's new version), will help ensure your dependencies can be met but the risk of third-party breakages is reduced. In production deployment, exact pinning and dedicated repositories are strongly recommended - some language tools do this in a more streamlined way (package.lock for npm, for example). However... narrowly pinning development code can prevent security patches being brought in, or conflicts when your module is later used with other code - the dependency versions conflict.

## pytest

This is the first contact with `pytest`. It automatically seeks out files starting with `test_` -- I'll walk you through `test_levenshtein.py` now...

You can run these tests from the command line:

### Exercise: Gene Hacking

In [ ]:
pip install py==1.11.0 --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 1.7 MB/s eta 0:00:00


In [ ]:
!pytest test_levenshtein.py

============================= test session starts ==============================
platform linux -- Python 3.11.13, pytest-7.4.4, pluggy-1.6.0
benchmark: 3.2.3 (defaults: timer=time.perf_counter disable_gc=False min_rounds=5 min_time=0.000005 max_time=1.0 calibration_precision=10 warmup=False warmup_iterations=100000)
rootdir: /content
plugins: benchmark-3.2.3, anyio-4.10.0, typeguard-4.4.4, langsmith-0.4.13
collected 5 items                                                              

test_levenshtein.py .....                                                [100%]

============================== 5 passed in 0.14s ===============================


There is a link there to Wikipedia, which has a standard textbook description of two algorithms - we will test which is faster. Can you fill in `my_levenshtein.py` to implement the _recursive_ `calculate_levenshtein` algorithm? Re-run the `pytest` command above until it passes!

### Extension: Gene Wilder

Going one further, can you implement the matrix version, using numpy, in the routine below? (algorithm also available from the same source)

### Extension: Gene E Us

Can you write a version that passes all the tests, but does not work in general? Can you add a test to catch your "mistake"? How much more robust can you make the testing?

This highlights a challenge with testing numerical or ML algorithms - that enumerating all possible cases is not necessarily possible. How might you break it up your testing, or your algorithm, to be able to more reliably test it?

## pylint

Pylint exists to help make sure your code is compliant with the PEP8 style guide (and a few others) - you can run it like so:

In [ ]:
!pylint my_levenshtein.py

/bin/bash: line 1: pylint: command not found


It's a pain, but once your code passes it, it's a breeze! It is recommended to include it in a continuous integration pipeline, just like testing - some developers include it in the githooks (automated code that is run when code is either committed locally, or pushed remotely).

### Exercise: Code Hy-gene

Try to adapt your code until it passes. Are all the checks useful? Are there some you would switch off?

In [ ]:
pip install hypothesis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.9/529.9 kB 6.8 MB/s eta 0:00:00


In [ ]:
from hypothesis import given
import hypothesis.strategies as st
import my_levenshtein
import Levenshtein as levenshtein_module  # known correct library

# Hypothesis will give us MANY random pairs of strings
@given(st.text(), st.text())
def test_levenshtein_matches_library(a, b):
    # Your function's result
    my_result = my_levenshtein.calculate_levenshtein(a, b)

    # Trusted "gold standard" result from python-Levenshtein
    expected = levenshtein_module.distance(a, b)

    # The property: They should always match
    assert my_result == expected


In [ ]:
!pytest -v

============================= test session starts ==============================
platform linux -- Python 3.11.13, pytest-7.4.4, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
hypothesis profile 'default'
benchmark: 3.2.3 (defaults: timer=time.perf_counter disable_gc=False min_rounds=5 min_time=0.000005 max_time=1.0 calibration_precision=10 warmup=False warmup_iterations=100000)
rootdir: /content
plugins: hypothesis-6.138.0, benchmark-3.2.3, anyio-4.10.0, typeguard-4.4.4, langsmith-0.4.13
collected 5 items                                                              

test_levenshtein.py::test_same_string_has_zero_distance PASSED           [ 20%]
test_levenshtein.py::test_different_strings_have_correct_distance PASSED [ 40%]
test_levenshtein.py::test_asdf_asfd_distance_is_two PASSED               [ 60%]
test_levenshtein.py::test_asf_asfd_distance_is_one PASSED                [ 80%]
test_levenshtein.py::test_amazon_aazonia_distance_is_three PASSED        [100%]

==============

Levenshtein distance was implemented in recursive and iterative forms.
Tests included fixed cases and property-based testing with Hypothesis.
Property-based testing helps catch issues that manual cases might miss